In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import os
from collections import Counter

filename = "/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv"
df = pd.read_csv(filename, usecols=['text', 'label'], dtype={'text': 'str', 'label': 'int32'})
dfX = df.loc[:, 'text']
dfY = df.loc[:, 'label']
dfX = dfX[:5000]
dfY = dfY[:5000]

vocabulary = Counter()
for title in dfX:
    words = title.split()
    validWords = filter(lambda x: len(x) <= 14, words)
    vocabulary.update(validWords)

# Truncate vocabulary
vocab_size = 1000
truncatedVocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

# Print out the 10 most common words and the number of times they occur
print("Most common words:", vocabulary.most_common()[:10])

# Convert words to tensor
words = tf.constant(truncatedVocabulary)

# Assign each word an ID
word_ids = tf.range(len(truncatedVocabulary), dtype=tf.int64)

# Create KeyValueTensor
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)

# Create lookup table
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

# Test the lookup table
testArr = "China and Iraq are in the dataset".split()
testRes = table.lookup(tf.constant(testArr))
print("Test result:", testRes)

Most common words: [('the', 56278), ('a', 31077), ('and', 30115), ('of', 28151), ('to', 26248), ('is', 20373), ('in', 17124), ('I', 13065), ('that', 12535), ('this', 11486)]
Test result: tf.Tensor([1377    2 1653   21    6    0 1677], shape=(7,), dtype=int64)



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [2]:
# Convert sentences to arrays of word ids
data = []
for title in dfX:
    sample = tf.strings.split(title)
    processed = table.lookup(sample)
    data.append(processed.numpy())

# Create a ragged tensor and then convert it to a padded dense tensor
ragged = tf.ragged.constant(data)
ragged = ragged.to_tensor(default_value=0)
#print(ragged)
#print(ragged.shape)

# Make dataset
features = tf.constant(ragged)
#print(features.shape)
labels = tf.constant(dfY)
#print(labels.shape)
train_set = tf.data.Dataset.from_tensor_slices((features, labels)).batch(32)
#print(next(train_set.batch(32).as_numpy_iterator())[0][0])


In [3]:
# Create model
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, batch_size=32, epochs=5)

Epoch 1/5


2022-01-30 06:00:09.134726: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


157/157 [==============================] - 532s 3s/step - loss: 0.6935 - accuracy: 0.5160
Epoch 2/5
157/157 [==============================] - 514s 3s/step - loss: 0.6930 - accuracy: 0.5136
Epoch 3/5
157/157 [==============================] - 521s 3s/step - loss: 0.6929 - accuracy: 0.5164
Epoch 4/5
157/157 [==============================] - 517s 3s/step - loss: 0.6928 - accuracy: 0.5164
Epoch 5/5
157/157 [==============================] - 519s 3s/step - loss: 0.6928 - accuracy: 0.5164


In [4]:
testArr = tf.strings.split(b"What the hell")
test = table.lookup(testArr)
print("testing: ", test)
prediction = model.predict(test)
print("prediction:", prediction)
print(prediction.shape)
print("avg:", np.average(prediction, axis=0))

testing:  tf.Tensor([259   0 823], shape=(3,), dtype=int64)
prediction: [[0.48791385]
 [0.4883399 ]
 [0.4888257 ]]
(3, 1)
avg: [0.4883598]
